# Assignment 3:  Using JOIN / Daily Show Guests

## Has the same guest appeared on the Daily Show under more than one job title or profession?

From 1999 to 2015 [Jon Stewart](https://en.wikipedia.org/wiki/Jon_Stewart) hosted [The Daily Show](https://en.wikipedia.org/wiki/The_Daily_Show).  The lists of guests that appreared on that show is available in a CSV file

Each entry has the form:

```2015,actor,4/15/15,Acting,Billy Crystal
```

The SQL file `jon_load.sql` defines the following table and loads the contents of the file into that table. 

In [1]:
%load_ext sql
%sql postgresql://isdb@localhost/postgres

'Connected: isdb@postgres'

Over the many years of its run under the stewardship of Jon Stewart, the Daily Show, has had a number of guests.  It is easy to determine the number of unique guests with

In [2]:
%%sql

select count( distinct name)
from Daily_Show_Guests

 * postgresql://isdb@localhost/postgres
1 rows affected.


count
1669


Of these 1669 guests, who has appeared the most?  Based on our class and lab discussions this should be straightforward:

In [3]:
%%sql

  SELECT name, count(name)
    FROM Daily_Show_Guests
GROUP BY name, job_title
ORDER BY count(name) DESC
   LIMIT 5;

 * postgresql://isdb@localhost/postgres
5 rows affected.


name,count
Fareed Zakaria,19
Denis Leary,17
Brian Williams,16
Paul Rudd,13
Ricky Gervais,13


A question I wondered about was "has anyone appeared on the Daily Show under different titles"?  For example, has a person who appeared once as a senator re-appeared as a governor or so.

If you calculate the number of unique `name`, `job_title` combinations you will get 1696. (You don't need to write the query for this.)

In [4]:
%%sql

   SELECT count(name) from
      (   SELECT DISTINCT name, job_title
            FROM Daily_Show_Guests
        GROUP BY name, job_title
        ORDER BY name) as distinct_name_title

 * postgresql://isdb@localhost/postgres
1 rows affected.


count
1696


So it seems that 1696 - 1669 = 27 people may have appeared on the Daily Show with more than one job title.

Is that really the case?

Write a pair of queries that will produce the below unique pairing of people who have appeared more than once with their job titles. 

Unfortunately, these is nobody with the characteristic I searched for. Rather we see dual entries with different job titles when that episode of the DS had two guests from two different professions.

The closest to having a guest appear with two different titles is Jon Favreau. But in this case we are refering to two different people all together. https://en.wikipedia.org/wiki/Jon_Favreau https://en.wikipedia.org/wiki/Jon_Favreau_(speechwriter)

Still this is an interesting pair of queries to write.

### The Queries

Based on what we've discussed in class draw some diagrams and try to reason a solution.

If need be, you can take a look at the guidlines I've provided below, but do give it a sincere try first.

To look at my hint, remove the leading `#` mark in the below line and evaluate it with a shift-return

In [5]:
#print('\n\n.tuokrow ot evah lliw uoy sliated wef a era erehT .cigol eht fo ecnesse eht si evoba ehT\n\n.elbat SD lanigiro eht htiw siht NIOJ neht dna boB ylliB tsuj teg ew 1 naht erom tnuoc a htiw seirtne rof gniretliF\n       \n1   eoD nhoJ             \n2  boB ylliB             \nspuorg owt teg ew        \n puorg hcae fo ezis eht gnitnuoc dna ,eman yb  gnipuorg ,a.1 morf seman eht tsuj gnitceleS b.1    \n)nwo sti no puorg a wor hcae( spuorg 3 teg dluow ew eltit boj ,eman yb GNIPUORG a.1    \n\n ```\nronrevoG   ,boB ylliB\nrotcA    ,eoD nhoJ\nrotaneS   ,boB ylliB\n```\n\n:senil gniwollof eht gnola nwo ruoy fo elpmaxe na tuo krow oD\n    \ntsil derised lanif eht teg ot elbat SD lanigiro eht htiw NIOJ dna 1yreuq morf seman fo tsil eht esU .2    \n        \nelbat siht ni seirtne 72 teg dluohs uoY        \nyrtne eno naht erom gnivah sPUORG yb retlif dna eman no elbat siht PUORG woN  .b           \n  .eman eht tsuj TCELES neht dna eltit_boj + eman yb PUORG  .a           \n  .seltit boj tnereffid rednu SD eht no deraeppa evah ohw stseug fo seman eht tsuj dniF  .1    \n    \nmelborp siht hcaorppa ot yaw a si gniwolloF\n\n.cigol dedeen eht erutpac taht selbat / smargaid emos ward ot TOL a pleh lliw ti edoc yna gnitirw erofeB\n'[::-1])

In [6]:
%%sql
-- Solution
-- Query 1 which will be stored in an intermediate table
-- Overal query seems to be create a table of guests that have multiple jobs 

DROP TABLE IF EXISTS Names;  


CREATE TABLE Names AS
    SELECT DISTINCT name
      FROM Daily_Show_Guests
     GROUP BY name
    HAVING count(DISTINCT job_title) > 1;

    
    
    
    
    
    

 * postgresql://isdb@localhost/postgres
Done.
27 rows affected.


[]

Joining back to the `Daily_Show_Guests` table you should get 54 rows (2*27)

In [7]:
%%sql
-- Solution

SELECT DISTINCT d.name, d.job_title
  FROM Daily_Show_Guests AS d
  JOIN Names AS n
    ON n.name = d.name
 ORDER BY d.name, d.job_title ASC;
 

  
    
    
    
    
    
    
    

 * postgresql://isdb@localhost/postgres
54 rows affected.


name,job_title
Adam Sandler and Chris Rock,Actor
Adam Sandler and Chris Rock,Comedian
Amy Yates Wuelfing & Gibby Haynes,Author
Amy Yates Wuelfing & Gibby Haynes,Musician
Bethany McLean and Joe Nocera,Editor
Bethany McLean and Joe Nocera,Journalist
Bob Odenkirk & David Cross,Actor
Bob Odenkirk & David Cross,Stand-Up Comedian
Bruce McCulloch and Mark McKinney,Actor
Bruce McCulloch and Mark McKinney,Comedian


```
+-----------------------------------------------------+-----------------------------------------+
|                         name                        |                job_title                |
+-----------------------------------------------------+-----------------------------------------+
|             Adam Sandler and Chris Rock             |                  Actor                  |
|             Adam Sandler and Chris Rock             |                 Comedian                |
|          Amy Yates Wuelfing & Gibby Haynes          |                  Author                 |
|          Amy Yates Wuelfing & Gibby Haynes          |                 Musician                |
|            Bethany McLean and Joe Nocera            |                  Editor                 |
|            Bethany McLean and Joe Nocera            |                Journalist               |
|              Bob Odenkirk & David Cross             |                  Actor                  |
|              Bob Odenkirk & David Cross             |            Stand-Up Comedian            |
|          Bruce McCulloch and Mark McKinney          |                  Actor                  |
|          Bruce McCulloch and Mark McKinney          |                 Comedian                |
|           Bruce Springsteen & Frank Caruso          |               Illustrator               |
|           Bruce Springsteen & Frank Caruso          |                 Musician                |
|               David Cross Bob Odenkirk              |                  Actor                  |
|               David Cross Bob Odenkirk              |            Stand-Up Comedian            |
|        Dr. Mustafa Barghouti and Anna Baltzer       |                 Activist                |
|        Dr. Mustafa Barghouti and Anna Baltzer       |              Public Speaker             |
|          Eduardo Sanchez and Daniel Myrick          |                 Director                |
|          Eduardo Sanchez and Daniel Myrick          |              Film Director              |
|    Frank DeCaro's Oscar Special, John Larroquette   |                  Actor                  |
|    Frank DeCaro's Oscar Special, John Larroquette   |                  Writer                 |
|         Hootie & the Blowfish, Billy Crystal        |                  Actor                  |
|         Hootie & the Blowfish, Billy Crystal        |                Rock Band                |
|            Hugh Hefner and Bentley twins            |                  Model                  |
|            Hugh Hefner and Bentley twins            |                Publisher                |
|             Jimmy Kimmel & Adam Carolla             |                 Comedian                |
|             Jimmy Kimmel & Adam Carolla             |             Television Host             |
|                     Jon Favreau                     |                  Actor                  |
|                     Jon Favreau                     |               Speechwriter              |
|           Kambiz Hosseini and Saman Arbabi          |                Journalist               |
|           Kambiz Hosseini and Saman Arbabi          |            Political Satirist           |
|            Kathryn Bigelow & Juan Zarate            |                Filmmaker                |
|            Kathryn Bigelow & Juan Zarate            |           White House Official          |
|         Katie Dellamaggiore and Pobo Efekoro        |               Chess Player              |
|         Katie Dellamaggiore and Pobo Efekoro        |              Film Director              |
|            Landon Donovan and Bob Bradley           |              Soccer Manager             |
|            Landon Donovan and Bob Bradley           |              Soccer Player              |
| Larry Lawton (skit)[4] and Jeff Sharlet (interview) |                  Author                 |
| Larry Lawton (skit)[4] and Jeff Sharlet (interview) |                Journalist               |
|         Maziar Bahari & Gael Garcí_a Bernal         |                Film Actor               |
|         Maziar Bahari & Gael Garcí_a Bernal         |                Journalist               |
|           Paul Teutul Sr. Paul Teutul Jr.           |                 Designer                |
|           Paul Teutul Sr. Paul Teutul Jr.           |           Television Producer           |
|               Posh Spice & Baby Spice               |              Businesswoman              |
|               Posh Spice & Baby Spice               |                  Singer                 |
|         Ringo Starr and The Ben Harper Band         |                 Musician                |
|         Ringo Starr and The Ben Harper Band         |                Rock Band                |
|              Robert Reich and Ben Stein             | Former United States Secretary of Labor |
|              Robert Reich and Ben Stein             |                  Writer                 |
|           Warren Buffett and Carol Loomis           |             Business Magnate            |
|           Warren Buffett and Carol Loomis           |                Journalist               |
|              William Weld & Al Sharpton             |     Former Govrnor Of Masssachusetts    |
|              William Weld & Al Sharpton             |                 Minister                |
|     Zach the Erect" Galifianakis & Will Ferrell"    |                  Actor                  |
|     Zach the Erect" Galifianakis & Will Ferrell"    |            Stand-up comedian            |
+-----------------------------------------------------+-----------------------------------------+
```